In [2]:
import findspark
findspark.init("/usr/hdp/current/spark2-client/")
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("PipelineOps") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

In [3]:
df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("hdfs:///user/admin/erkan/adult_preprocessed.csv") \

In [4]:
df.toPandas().head()

,workclass,education,education_merged,marital_status,occupation,relationship,race,sex,native_country,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,output
0,State-gov,Bachelors,Under-Graduate,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,39,77516.0,13.0,2174.0,0.0,40.0,<=50K
1,Self-emp-not-inc,Bachelors,Under-Graduate,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,50,83311.0,13.0,0.0,0.0,13.0,<=50K
2,Private,HS-grad,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,38,215646.0,9.0,0.0,0.0,40.0,<=50K
3,Private,11th,High-School,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,53,234721.0,7.0,0.0,0.0,40.0,<=50K
4,Private,Bachelors,Under-Graduate,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,28,338409.0,13.0,0.0,0.0,40.0,<=50K


In [5]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, StandardScaler

# StringIndexer

In [6]:
workclass_indexer = StringIndexer() \
.setInputCol("workclass") \
.setOutputCol("workclass_index") \
.setHandleInvalid("skip")

In [7]:
education_merged_indexer = StringIndexer() \
.setInputCol("education_merged") \
.setOutputCol("education_merged_index") \
.setHandleInvalid("skip")

In [8]:
marital_status_indexer = StringIndexer() \
.setInputCol("marital_status") \
.setOutputCol("marital_status_index") \
.setHandleInvalid("skip")

In [9]:
occupation_indexer = StringIndexer() \
.setInputCol("occupation") \
.setOutputCol("occupation_index") \
.setHandleInvalid("skip")

In [10]:
relationship_indexer = StringIndexer() \
.setInputCol("relationship") \
.setOutputCol("relationship_index") \
.setHandleInvalid("skip")

In [11]:
race_indexer = StringIndexer() \
.setInputCol("race") \
.setOutputCol("race_index") \
.setHandleInvalid("skip")

In [12]:
native_country_indexer = StringIndexer() \
.setInputCol("native_country") \
.setOutputCol("native_country_index") \
.setHandleInvalid("skip")

In [13]:
encoder = OneHotEncoderEstimator() \
.setInputCols(["workclass_index","education_merged_index","marital_status_index","occupation_index","relationship_index",
               "race_index", "native_country_index"]) \
.setOutputCols(["workclass_index_enc","education_merged_index_enc","marital_status_index_enc","occupation_index_enc",
                "relationship_index_enc", "race_index_enc", "native_country_index_enc"])

In [14]:
tum_nitelikler = ["workclass_index_enc","education_merged_index_enc","marital_status_index_enc","occupation_index_enc",
                "relationship_index_enc", "race_index_enc", "native_country_index_enc","age","fnlwgt","education_num","capital_gain","capital_loss","hours_per_week"]

In [15]:
tum_nitelikler

['workclass_index_enc',
 'education_merged_index_enc',
 'marital_status_index_enc',
 'occupation_index_enc',
 'relationship_index_enc',
 'race_index_enc',
 'native_country_index_enc',
 'age',
 'fnlwgt',
 'education_num',
 'capital_gain',
 'capital_loss',
 'hours_per_week']

In [16]:
assembler = VectorAssembler() \
.setInputCols(tum_nitelikler) \
.setOutputCol("vectorized_features")

In [17]:
label_indexer = StringIndexer() \
.setInputCol("output") \
.setOutputCol("label")

In [18]:
scaler = StandardScaler() \
.setInputCol("vectorized_features") \
.setOutputCol("features")

In [19]:
from pyspark.ml.classification import LogisticRegression

In [20]:
lr_object = LogisticRegression() \
.setFeaturesCol("features") \
.setLabelCol("label") \
.setPredictionCol("prediction")

In [21]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=142)

In [22]:
from pyspark.ml import Pipeline

In [23]:
pipeline_nesnesi = Pipeline() \
.setStages([workclass_indexer, education_merged_indexer, marital_status_indexer, occupation_indexer, relationship_indexer, 
            race_indexer, native_country_indexer, encoder, assembler, label_indexer, scaler, lr_object])

In [24]:
pipeline_model = pipeline_nesnesi.fit(train_df)

In [25]:
sonuc_df = pipeline_model.transform(test_df).select("label","prediction")
sonuc_df.toPandas().head()

,label,prediction
0,0.0,0.0
1,0.0,1.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [26]:
from sklearn.metrics import accuracy_score

In [27]:
y_pred = sonuc_df.select("prediction").toPandas()
y_true = sonuc_df.select("label").toPandas()

In [28]:
accuracy_score(y_true, y_pred)

0.8471119929453262